In [1]:
cd ..

/home/moses/galvanize/gimme-bus


In [2]:
ls

app/   nodes.txt  reference.txt  scratch_notebooks/
code/  README.md  schedule.pkl


In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import cPickle as pickle
from code import utilities as ut
from code import schedule

In [4]:
gtfs_dir = '../bus_project_data/google_transit/'

In [6]:
sched = schedule.Schedule(gtfs_dir)

In [8]:
sched.stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,6680994,07:09:00,07:09:00,4015,1,,,,
1,6680994,07:10:08,07:10:08,6294,2,,,,
2,6680994,07:11:21,07:11:21,6290,3,,,,
3,6680994,07:12:00,07:12:00,6314,4,,,,
4,6680994,07:12:47,07:12:47,6307,5,,,,


In [11]:
nodes = []
for val in sched.all_stop_timepoints.itervalues():
        nodes.extend(val)

In [18]:
with open('nodes.txt', 'w') as f:
    f.write('\n'.join(nodes))

In [42]:
G = nx.DiGraph()

In [43]:
G.add_nodes_from(nodes)

In [33]:
nodes[155][:-9]

'390'

In [35]:
nodes[155][-8:]

'12:46:00'

In [44]:
for i, node in enumerate(nodes):
    if i == 0:
        last_node = node
        continue
    node_id = node[:-9]
    node_time = node[-8:]
    last_node_id = last_node[:-9]
    last_node_time = last_node[-8:]
    wait_time = ut.diff_timestamps(last_node_time, node_time)
    if node_id == last_node_id:
        G.add_edge(last_node, node, duration=wait_time, type='wait')
    last_node = node

In [40]:
ut.diff_timestamps(nodes[155][-8:], nodes[156][-8:])

480

In [45]:
G.number_of_edges()

737588

In [46]:
G.edge

{'5851_08:11:54': {'5851_08:12:54': {'duration': 60, 'type': 'wait'}},
 '3218_08:00:01': {'3218_08:00:13': {'duration': 12, 'type': 'wait'}},
 '6086_17:24:00': {'6086_17:36:00': {'duration': 720, 'type': 'wait'}},
 '5651_16:53:18': {'5651_16:53:20': {'duration': 2, 'type': 'wait'}},
 '5646_13:25:11': {'5646_13:27:58': {'duration': 167, 'type': 'wait'}},
 '6772_18:43:31': {'6772_18:44:31': {'duration': 60, 'type': 'wait'}},
 '4302_10:54:35': {'4302_10:55:11': {'duration': 36, 'type': 'wait'}},
 '3055_04:24:27': {'3055_04:44:27': {'duration': 1200, 'type': 'wait'}},
 '4303_21:36:44': {'4303_21:40:44': {'duration': 240, 'type': 'wait'}},
 '4798_22:48:17': {'4798_22:50:44': {'duration': 147, 'type': 'wait'}},
 '4213_07:35:26': {'4213_07:37:26': {'duration': 120, 'type': 'wait'}},
 '7188_09:55:44': {'7188_10:01:44': {'duration': 360, 'type': 'wait'}},
 '6994_08:45:37': {'6994_08:46:27': {'duration': 50, 'type': 'wait'}},
 '3288_16:38:01': {'3288_16:42:37': {'duration': 276, 'type': 'wait'}}

In [49]:
sched.stops.index

Int64Index([ 390,  913, 3003, 3004, 3008, 3009, 3010, 3011, 3013, 3015,
            ...
            7803, 7804, 7805, 7806, 7807, 7808, 7809, 7815, 7821, 7841],
           dtype='int64', name=u'stop_id', length=3561)

In [59]:
hm = {}
for row in sched.stops.iterrows():
    stop_id = row[0]
    stop_info = row[1]  
    # calculate distance between this stop and every other stop
    dists = sched.stops.apply(lambda x: ut.haversine( (x['stop_lon'], \
                        x['stop_lat']), (stop_info['stop_lon'],\
                        stop_info['stop_lat'])), axis=1)
    # only keep the stops within max_dist
    dists = dists[dists <= 0.2]

    # don't connect the stop to itself, duh
    dists = dists.drop(stop_id)
    hm[stop_id] = dists.to_dict()

In [61]:
len(hm.keys())

3561

In [62]:
sched.stops.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3561 entries, 390 to 7841
Data columns (total 6 columns):
stop_name    3561 non-null object
stop_desc    3561 non-null object
stop_lat     3561 non-null float64
stop_lon     3561 non-null float64
zone_id      3561 non-null object
stop_url     3561 non-null object
dtypes: float64(2), object(4)
memory usage: 194.7+ KB


In [63]:
hm[4555]

{3122: 0.15504650921473909,
 3123: 0.16261322987961688,
 4547: 0.14847919196425494,
 5509: 0.19929198529797293,
 5510: 0.18017153035422057,
 7077: 0.19941701455171287,
 7170: 0.15835215535088318,
 7352: 0.15855707843180977,
 7365: 0.16710903667876204,
 7428: 0.089089492511880017}

In [14]:
G_test = nx.read_edgelist('edge_test.txt')

In [26]:
G_test['1']['7']['beans']

3